In [1]:
import sys
!{sys.executable} -m pip install mlxtend

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Question 1

In [2]:
data = pd.read_csv('mammographic_masses.csv')

In [6]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets

,support,itemsets


In [7]:

a=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

a[["antecedents","consequents","support","confidence"]]

ValueError: The input DataFrame `df` containing the frequent itemsets is empty.

# Question 2

# Question 3

# Question 4

# Question 5